In [1]:
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import os
from openai import OpenAI
import json 

In [2]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

In [9]:
# configs
load_dotenv()
api_key = os.getenv('YANDEX_API_KEY')
folder_id = os.getenv('FOLDER_ID')

### Определение класса агента

Данный класс способен принимать на вход различные инструменты в виде python функций, а также создавать и пополнять векторную базу данных для настройки RAG.

In [19]:
import os


# model = f"gpt://{folder_id}/yandexgpt/latest"
model = f"gpt://{folder_id}/gemma-3-27b-it/latest"
# model = f"gpt://{folder_id}/gpt-oss-120b/latest"
# model = f"gpt://{folder_id}/qwen2-vl-7b-instruct"

client = OpenAI(
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

In [20]:
prompt = "Привет"

res = client.responses.create(
    model = model,
    input = [
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
            ],
        }
    ],
)

print(res.output_text)

Привет! Чем могу помочь? Что тебя интересует?



In [21]:
import io

class Agent():

    def __init__(self,
            name,
            instruction, 
            tools = [], search_content = [], 
            model = model,
            response_format = None
            ):
        self.user_sessions = {}
        self.name = name
        self.instruction = instruction
        self.model = model
        self.tool_map = { x.__name__ : x for x in tools if issubclass(x, BaseModel) }
        self.tools = [
            self._create_tool_annot(x) for x in tools
        ]
        self.response_format = response_format
        self.vector_store = None
        if search_content:
            i=0
            self.vector_store = client.vector_stores.create(name=f'rag_store_{self.name}')
            for c in search_content:
                f = client.files.create(
                        purpose="assistants",
                        file = (f'rag_{self.name}_{i}.txt',io.BytesIO(c.encode("utf-8")),'text/markdown'))
                client.vector_stores.files.create(file_id=f.id, vector_store_id=self.vector_store.id)
                print(f" + Uploading rag_{self.name}_{i}.txt as id={f.id} to store={self.vector_store.id}")
                i+=1
            self.tools.append({
                "type" : "file_search",
                "vector_store_ids" : [self.vector_store.id],
                "max_num_results" : 5,
            })
            
    def _create_tool_annot(self, x):
        if issubclass(x, BaseModel):
            return {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            }
        else:
            return x

    def __call__(self, message, session_id='default',return_raw=False):
        s = self.user_sessions.get(session_id,{ 'previous_response_id' : None, 'history' : [] })
        s['history'].append({ 'role': 'user', 'content': message })
        txt = None
        if self.response_format:
            txt = {
                "format" : {
                    "type" : "json_schema",
                    "name" : "struct_out",
                    "schema" : self.response_format.model_json_schema()
                }
            }
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            instructions = self.instruction,
            previous_response_id = s['previous_response_id'],
            input = message,
            text = txt
        )
        # Обрабатываем вызов локальных инструментов
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                print(f" + Обрабатываем: {call.name} ({call.arguments})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(call.arguments))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                #print(f" + Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        # MCP Approval Requests
        mcp_approve = [ item for item in res.output if item.type == "mcp_approval_request"]
        if mcp_approve:
            res = client.responses.create(
                model=self.model,
                previous_response_id=res.id,
                tools = self.tools,
                input=[{
                    "type": "mcp_approval_response",
                    "approve": True,
                    "approval_request_id": m.id
                }
                for m in mcp_approve
                ])
        s['previous_response_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        self.user_sessions[session_id] = s
        if return_raw:
            return res
        if self.response_format:
            return self.response_format.model_validate_json(res.output_text)
        else:
            return res.output_text

    def history(self, session_id='default'):
        return self.user_sessions[session_id]['history']

### Агент-консультант СЛФ

В примере будет использоваться каталог СЛФ для создания агента, который способен ориентироваться в каталоге. Вместо файла каталога можно использоваться другие документы: нормативные акты, учебники, датасеты

In [28]:
# сохраняем файл, который будем использовать для RAG, в формате markdown
catalog = pd.read_parquet('./nomen.parquet')
catalog[['name', 'article']].to_markdown('nomen.md')

In [29]:
# здесь указывается путь к файлу с каталогом в формате markdown
with open("./nomen.md", encoding="utf-8") as f:
    food_wine = f.readlines()
header = food_wine[:2]
chunk_size = 1000 * 3  # approx 1000 tokens * 3 char/token
docs = []
s = header.copy()
for x in food_wine[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        docs.append("".join(s))
        s = header.copy()

In [30]:
prompt = f"""
Ты - консультант на складе поставщика продуктов для сегмента HoReCa, который должен советовать клиентам продукты и отвечать 
на вопросы по ассортименту продуктов, используя имеющуюся у тебя информацию. Если в каталоге нет указанного продукта
 - напиши, что продукт отсутствует в каталоге. Не придумывай ничего!
"""

consultant = Agent(
    'consultant',
    search_content=docs,
    instruction=prompt) 

 + Uploading rag_consultant_0.txt as id=fvtkfauihf7gq40p314b to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_1.txt as id=fvt1e738167qgv6or80k to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_2.txt as id=fvtovpn31bc6urhq9va9 to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_3.txt as id=fvtp6fh4dc6f5omvv4tm to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_4.txt as id=fvti55e4hifoeltblmrb to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_5.txt as id=fvt1h4kvod1ra74df0dm to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_6.txt as id=fvtqssebhrvvj7ek11d3 to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_7.txt as id=fvtabj1lrhtgfn68h9nk to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_8.txt as id=fvtvo2kgf2v7msjsgb53 to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_9.txt as id=fvtahj8j07s4uhp72h96 to store=fvtovs52bc72rmqrdeg1
 + Uploading rag_consultant_10.txt as id=fvtf9lrh1p26m6uvjidq to store=fvtovs52bc72rmqrdeg

In [31]:
printx(consultant("Что ты умеешь?"))

Я – консультант на складе поставщика продуктов для сегмента HoReCa. Я могу давать советы по продуктам и отвечать на вопросы об ассортименте, используя информацию из каталога. Если продукта нет в каталоге, я сообщу об этом. Для поиска информации я могу использовать функцию поиска по каталогу.


In [32]:
printx(consultant("Посоветуй продукты из каталога, которые можно использовать для приготовления блюда 'Борщ'"))

Для приготовления борща из нашего каталога вам могут понадобиться следующие продукты:

*   **Заправка для Борща Dinner Service** (артикул 115873)
*   **Заправка для Борща Bravolli Professional** (артикул 116477)
*   **Набор для борща зам. Мираторг** (артикул 116446)
*   **Свекла**, к сожалению, отсутствует в каталоге.
*   **Капуста**, к сожалению, отсутствует в каталоге.
*   **Картофель**, к сожалению, отсутствует в каталоге.
*   **Мясо** (например, говядина или свинина), к сожалению, отсутствует в каталоге.
*   **Томатная паста** (артикул 111948)
*   **Перец черный горошек** (артикул 27117)
*   **Масло подсолнечное** (артикул 112401)